## 第一部分：了解 nn.Module的基本操作

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
import torch.nn.functional as F
import numpy as np

In [2]:
model = models.resnet18()

### 打印出 model底下所有 parameters 的 name 以及對應的 shape 

In [3]:
for name, param in model.named_parameters():
    print(name, param.shape)

conv1.weight torch.Size([64, 3, 7, 7])
bn1.weight torch.Size([64])
bn1.bias torch.Size([64])
layer1.0.conv1.weight torch.Size([64, 64, 3, 3])
layer1.0.bn1.weight torch.Size([64])
layer1.0.bn1.bias torch.Size([64])
layer1.0.conv2.weight torch.Size([64, 64, 3, 3])
layer1.0.bn2.weight torch.Size([64])
layer1.0.bn2.bias torch.Size([64])
layer1.1.conv1.weight torch.Size([64, 64, 3, 3])
layer1.1.bn1.weight torch.Size([64])
layer1.1.bn1.bias torch.Size([64])
layer1.1.conv2.weight torch.Size([64, 64, 3, 3])
layer1.1.bn2.weight torch.Size([64])
layer1.1.bn2.bias torch.Size([64])
layer2.0.conv1.weight torch.Size([128, 64, 3, 3])
layer2.0.bn1.weight torch.Size([128])
layer2.0.bn1.bias torch.Size([128])
layer2.0.conv2.weight torch.Size([128, 128, 3, 3])
layer2.0.bn2.weight torch.Size([128])
layer2.0.bn2.bias torch.Size([128])
layer2.0.downsample.0.weight torch.Size([128, 64, 1, 1])
layer2.0.downsample.1.weight torch.Size([128])
layer2.0.downsample.1.bias torch.Size([128])
layer2.1.conv1.weight tor

### 為了使 forward propagation 加速 並降低 memory 使用量，請將所有 parameters 的requires_grad 關閉，關閉之後執行  forward propagation

In [4]:
input_ = torch.randn(1, 3, 128, 128)

In [5]:
for param in model.parameters():
    param.requires_grad = False

In [6]:
output = model(input_)
print(output.shape)

torch.Size([1, 1000])


## 第二部分：依照指令，以較簡潔的方式搭建出模型

* input_shape = torch.Size([10, 12])
* 先經過一層 nn.Linear(12, 10)
* 經過10層 nn.Linear(10, 10)
* 最後經過 nn.Linear(10, 3) 輸出
* 每一個 nn.Linear過完後要先經過 nn.BatchNorm1d 才能到下一層，輸出層不用


In [12]:
input_ = torch.randn(10, 12)
## 示範
Linear0 = nn.Linear(12, 10)
Linear1 = nn.Linear(10, 10)

BN = nn.BatchNorm1d(10)

In [13]:
layer_input = nn.Sequential(Linear0, BN)
layer_hidden = nn.Sequential(Linear1, BN)
layer_output = nn.Linear(10, 3)

layer = nn.ModuleList()
for i in range(10):
    layer.append(layer_hidden)


In [23]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.input = layer_input
        self.repeat_linear = layer
        self.output = layer_output

    def forward(self, x):
        x = self.input(x)
        
        for module in self.repeat_linear:
            x = module(x)
        
        x = self.output(x)
        return x

In [24]:
model = Model()

In [25]:
model

Model(
  (input): Sequential(
    (0): Linear(in_features=12, out_features=10, bias=True)
    (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (repeat_linear): ModuleList(
    (0): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (2): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (3): Sequential(
      (0): Linear(in_features=10, out_features=10, bias=True)
      (1): BatchNorm1d(10, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (4): Sequential(
      (0): Linear(in_features=10, out_feature

In [26]:
input_ = torch.randn(10,12)
outupt = model(input_)

In [27]:
outupt

tensor([[-0.1713,  0.1121,  0.2433],
        [ 0.0458, -0.0412,  0.0735],
        [ 1.5109, -0.1984, -0.1362],
        [-0.0200, -0.3222,  0.0387],
        [ 0.1261, -0.1159, -0.2353],
        [-1.0538, -0.3115, -0.0991],
        [-0.7312, -0.0949,  0.3169],
        [-0.2085, -0.1548, -0.1204],
        [ 1.1181,  0.0277, -0.4510],
        [ 1.9492, -0.1144, -0.0872]], grad_fn=<AddmmBackward>)